In [1]:
from gophish import Gophish
from gophish.models import SMTP, Page, Template, Campaign, User, Group
import json
import subprocess

In [ ]:
from email_generator import generate_email_content

In [5]:
def config_and_start(config_path):
    with open(config_path, "r") as file:
        config = json.load(file)
        GOPHISH_API_KEY = config.get("GOPHISH_API_KEY")
        email_pass = config.get("email_pass")
        email_address = config.get("email_address")
        host =  config.get("host")
        port = config.get("port")
        gophish_api = Gophish(GOPHISH_API_KEY, verify=False)
    
    return(email_address, email_pass, host, port, gophish_api)

In [6]:
email_address, email_pass, host, port, gophish_api = config_and_start("Alek_config.json")

In [7]:
def create_sending_profile(name):
    
    sending_profile = SMTP(
    name=name,
    from_address=email_address,
    from_name=name,
    reply_to_address=email_address,  
    host=host,  
    username=email_address,  
    password=email_pass,  
    port=587,  
    tls=True,  
    ssl=False,)

    gophish_api.smtp.post(sending_profile)


In [ ]:
create_sending_profile('Document Sign')

In [9]:
def clear_sending_profiles():
    for sending_profile in gophish_api.smtp.get():
        gophish_api.smtp.delete(sending_profile.id)

In [94]:
def html_reader(file_path):
    with open(file_path, "r") as file:
        html_str = file.read()
    return html_str


In [ ]:
html_reader("../web-pages/index.html")

In [10]:
def create_landing_page(name, html_str):
    landing_page = Page(
        name=name,
        redirect_url="https://alkszo.github.io/security-training/landing_page",
        html=html_str, 
        capture_credentials=True,
        capture_passwords=True 
    )

    gophish_api.pages.post(landing_page)

In [ ]:
create_landing_page("Scammer", html_reader("../web-pages/index.html"))
#"127.0.0.1:3333"


In [103]:
def clear_landing_pages():
    for slanding_page in gophish_api.pages.get():
        gophish_api.pages.delete(slanding_page.id)

In [ ]:
clear_landing_pages()

In [106]:
def create_email_template(name, subject, html_str):
    email_template = Template(
        name=name,
        subject=subject,
        html=html_str,    
    )

    gophish_api.templates.post(email_template)

In [ ]:
create_email_template('Scammer', 'Document Sign', html_reader('docu_sign_mail.html'))

In [108]:
def clean_email_templates():
    for email_template in gophish_api.templates.get():
        gophish_api.templates.delete(email_template.id)

In [ ]:
clean_email_templates()

In [111]:
def create_group(name, first_name, last_name, email, position):
    group = Group(
        name=name, 
        targets=[User(first_name=first_name, last_name=last_name, email=email, position=position)]
    )

    gophish_api.groups.post(group)

In [ ]:
create_group('group1', 'Alek', 'Szostakowski', 'aleksander.szostakowski@gmail.com', 'data janitor')

In [113]:
def clear_groups():
    for group in gophish_api.groups.get():
        gophish_api.groups.delete(group.id)

In [ ]:
clear_groups()

In [117]:
def create_campaign(name, template_name, page_name, sending_profile_name, group_name):
    campaign = Campaign(
    name=name,
    template=Template(name=template_name),
    page = Page(name=page_name),
    smtp=SMTP(name=sending_profile_name),
    groups=[Group(name=group_name)],
    url="http://127.0.0.1"
)
    
    gophish_api.campaigns.post(campaign)

In [ ]:
create_campaign('Scam', 'Scammer', 'Scammer', 'Document Sign', 'group1')